In [1]:
import osmnx as ox
import numpy as np
import pandas as pd


In [8]:
df = pd.read_csv('../gavi.csv')
df['LAT'] = np.nan
df['LONG'] = np.nan
count = 0
# Define the graph for Bengaluru
G = ox.graph_from_place('Bengaluru, India', network_type='all')

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Get the origin landmark for this row
    origin_landmark = row['O_LANDMARK']
    
    # Geocode the origin landmark to obtain its latitude and longitude coordinates
    try:
        origin_location = ox.geocode(origin_landmark)
        count+=1
    except Exception as e:
        print(f"Error geocoding {origin_landmark}: {e}")
        continue  # Skip to the next iteration if geocoding fails
    
    # Add the latitude and longitude coordinates to the DataFrame
    df.at[index, 'LAT'] = origin_location[0]
    df.at[index, 'LONG'] = origin_location[1]

Error geocoding Rajajinagar,1st N block,sardar Patel high school : Nominatim could not geocode query "Rajajinagar,1st N block,sardar Patel high school "
Error geocoding Rr nagar,gopalan mall: Nominatim could not geocode query "Rr nagar,gopalan mall"
Error geocoding Mahalaxmi,nandini layout : Nominatim could not geocode query "Mahalaxmi,nandini layout "
Error geocoding Peenya 2nd stage,anupama school : Nominatim could not geocode query "Peenya 2nd stage,anupama school "
Error geocoding Konankunte prestige falcon city: Nominatim could not geocode query "Konankunte prestige falcon city"
Error geocoding Madanayakanahalli, kudurigere colony : Nominatim could not geocode query "Madanayakanahalli, kudurigere colony "
Error geocoding Rajajinagar, akash tuition : Nominatim could not geocode query "Rajajinagar, akash tuition "
Error geocoding Nagadevanahalli ,gyanabarathi: Nominatim could not geocode query "Nagadevanahalli ,gyanabarathi"
Error geocoding Mahalaxmi,kurubruhalli: Nominatim could no

In [10]:
# Create a new column 'LAT' and 'LONG' in dataframe df
df = pd.read_csv('../gavi.csv')
df['LAT'] = np.nan
df['LONG'] = np.nan

# Define the graph for Bengaluru
G = ox.graph_from_place('Bengaluru, India', network_type='all')

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Get the origin landmark for this row
    origin_landmark = row['O_LANDMARK']
    
    # Geocode the origin landmark to obtain its latitude and longitude coordinates
    try:
        origin_location = ox.geocode(origin_landmark)
    except Exception as e:
        print(f"Error geocoding {origin_landmark}: {e}")
        continue  # Skip to the next iteration if geocoding fails
    
    # If geocoding returns None, find the nearest point on the nearest edge
    if origin_location is None:
        nearest_edge = ox.distance.nearest_edges(G, row['LAT'], row['LONG'])
        
        # Iterate over edges to find the nearest node on the nearest edge
        min_distance = float('inf')
        nearest_node = None
        for edge in nearest_edge:
            for u, v, _ in G.edges(keys=True, data=False):
                if (u, v) == edge or (v, u) == edge:
                    distance_to_u = ox.distance.euclidean_dist_vec(G.nodes[u]['y'], G.nodes[u]['x'], row['LAT'], row['LONG'])
                    distance_to_v = ox.distance.euclidean_dist_vec(G.nodes[v]['y'], G.nodes[v]['x'], row['LAT'], row['LONG'])
                    if distance_to_u < min_distance:
                        min_distance = distance_to_u
                        nearest_node = u
                    if distance_to_v < min_distance:
                        min_distance = distance_to_v
                        nearest_node = v
        
        # If nearest_node is still None, handle the case when no nearest node is found
        if nearest_node is None:
            print(f"No nearest node found for {origin_landmark}")
            continue
        
        # Get the coordinates of the nearest node
        origin_location = G.nodes[nearest_node]['y'], G.nodes[nearest_node]['x']
    
    # Add the latitude and longitude coordinates to the DataFrame
    df.at[index, 'LAT'] = origin_location[0]
    df.at[index, 'LONG'] = origin_location[1]

Error geocoding Rajajinagar,1st N block,sardar Patel high school : Nominatim could not geocode query "Rajajinagar,1st N block,sardar Patel high school "
Error geocoding Rr nagar,gopalan mall: Nominatim could not geocode query "Rr nagar,gopalan mall"
Error geocoding Mahalaxmi,nandini layout : Nominatim could not geocode query "Mahalaxmi,nandini layout "
Error geocoding Peenya 2nd stage,anupama school : Nominatim could not geocode query "Peenya 2nd stage,anupama school "
Error geocoding Konankunte prestige falcon city: Nominatim could not geocode query "Konankunte prestige falcon city"
Error geocoding Madanayakanahalli, kudurigere colony : Nominatim could not geocode query "Madanayakanahalli, kudurigere colony "
Error geocoding Rajajinagar, akash tuition : Nominatim could not geocode query "Rajajinagar, akash tuition "
Error geocoding Nagadevanahalli ,gyanabarathi: Nominatim could not geocode query "Nagadevanahalli ,gyanabarathi"
Error geocoding Mahalaxmi,kurubruhalli: Nominatim could no

In [14]:
import requests

def get_route_and_intersection_info(api_key, origin_coords, destination_coords):
    # Request route information from TomTom Routing API
    url = f"https://api.tomtom.com/routing/1/calculateRoute/{origin_coords['lat']},{origin_coords['lon']}:{destination_coords['lat']},{destination_coords['lon']}/json"
    params = {
        "key": "PafHsNp2wZ7OAXGNvoTGAeUoyRGY3EIz",
        "traffic": "true",
        "routeType": "fastest",
        "maxEdgesPerEdgeGroup": -1
    }
    response = requests.get(url, params=params)
    route_data = response.json()

    if 'routes' in route_data and len(route_data['routes']) > 0:
        route = route_data['routes'][0]
        intersections = get_number_of_intersections(api_key, route)
        avg_street_width = get_average_street_width(api_key, route)
        return route, intersections, avg_street_width
    else:
        print("No route found.")
        return None, None, None

# Assuming you have implemented the functions get_number_of_intersections and get_average_street_width as described earlier

# Example usage
api_key = "PafHsNp2wZ7OAXGNvoTGAeUoyRGY3EIz"
origin_coords = {"lat": 13.0093896138943, "lon":77.4975764983865}  
destination_coords = {"lat": 13.000381302677624, "lon": 77.55041494885654}  

route, intersections, avg_street_width = get_route_and_intersection_info(api_key, origin_coords, destination_coords)
if route is not None:
    print("Route:", route)
    print("Number of Intersections:", intersections)
    print("Average Street Width:", avg_street_width)


No route found.


In [9]:
, 

34